In [15]:
# %matplotlib ipympl
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import torch
import torch.nn.functional as F
import os
import psutil
from utils.preprocess import *

# Get available memory
available_memory = psutil.virtual_memory().available

# Assuming np.float64
element_size = np.dtype(np.float64).itemsize

# Calculate max array size
max_array_size = available_memory / element_size

print(f"Maximum array size (np.float64): {max_array_size} elements")
# from mpl_interactions import ipyplot as iplt
# from matplotlib.cm import ScalarMappable

Maximum array size (np.float64): 27943418880.0 elements


In [2]:
# Data loading Version 2 - toy data previously
data_root = '/scratch/bbsg/hangy6/RLS/data'
# patient_dir = f'{data_root}/patient01-08-27-2023'
# patient_dir = f'{data_root}/patient05-02-15-2024'
# patient_dir = f'{data_root}/patient06-02-17-2024'
# patient_dir = f'{data_root}/patient09-03-01-2024'
# patient_dir = f'{data_root}/patient11-03-15-2024'
patient_dir = f'{data_root}/patient13-03-31-2024'
# patient_dir = f'{data_root}/patient14-04-03-2024'

print(f'Memory available before loading data: {psutil.virtual_memory().available}')
try:
    del matrices, timestamps, data
except:
    pass

window_size = 156
data_file = os.path.join(patient_dir, 'raw_pressure_data.npy')
timestamps_file = os.path.join(patient_dir, 'raw_timestamps.npy')
if os.path.exists(data_file) and os.path.exists(timestamps_file):
    print('load existing pressure data...')
    matrices = np.load(data_file).astype(np.float32)
    timestamps = np.load(timestamps_file)
    print(f'successfully loaded pressure data from {data_file} and timestamps from {timestamps_file}')
else:
    print('Generating pressure data from raw json file...')
    pressure_data = load_json(os.path.join(patient_dir, 'raw_data.json'))
    matrices, timestamps = get_pressure_matrices(pressure_data)
    np.save(data_file, np.array(matrices))
    np.save(data_file, np.array(timestamps))
    print('Successfully saved pressure data and timestamps')

594890


In [ ]:
save_data = False
balanced = False
window_size = 40
    
EMG_label_path = f'{patient_dir}/positive_timestamps.csv'
train_data, train_label, train_roi_label, val_data, val_label, val_roi_label = \
make_labels(matrices, timestamps, EMG_label_path, filter_timestamp=None, \
            window_size=window_size, balanced=balanced, overlap_train=False, split_ratio=1.0, plot_statistics=False)
# make_labels(matrices, timestamps, EMG_label_path, filter_timestamp='2023-12-11T01:00:56.3789055-06:00', \
#             window_size=window_size, balanced=True, overlap_train=False, split_ratio=0.8)
if not balanced:
    window_size = f'{window_size}_LT'

positive_idx = train_label > 0
negative_idx = train_label == 0
positive_data = train_data[positive_idx]
negative_data = train_data[negative_idx]
batch1_positive = positive_data[:, ...]
# batch2_positive = positive_data[50:100, ...]
batch1_negative = negative_data[:, ...]
# batch2_negative = negative_data[25:100, ...]
batch1 = np.concatenate([batch1_positive, batch1_negative], axis=0)
# batch2 = np.concatenate([batch2_positive, batch2_negative], axis=0)
cosine_matrix = cosine_similarity(batch1, batch1)
plt.imshow(cosine_matrix)
plt.colorbar()
len_pos, N = len(positive_data), len(train_data)
# plt.plot([len_pos] * N, np.arange(N), color='red')
# plt.plot(np.arange(N), [len_pos] * N, color='red')
plt.savefig(f'{path}/win{window_size}_data_cosine_similarity.png')

if save_data:
    np.save(f'{patient_dir}/win{window_size}_sensing_mat_data_train.npy', train_data)
    np.save(f'{patient_dir}/win{window_size}_sensing_mat_data_val.npy', val_data)
    np.save(f'{patient_dir}/win{window_size}_EMG_label_train.npy', train_label)
    np.save(f'{patient_dir}/win{window_size}_EMG_roi_label_train.npy', train_roi_label)
    np.save(f'{patient_dir}/win{window_size}_EMG_label_val.npy', val_label)
    np.save(f'{patient_dir}/win{window_size}_EMG_roi_label_val.npy', val_roi_label)

(14872, 40, 16, 16)
timestamps at break: 2024-03-16T05:21:41.9322938-05:00
(14382, 40, 16, 16) 1141 (0, 40, 16, 16) 0


/tmp/ipykernel_3241946/3121324919.py:187: RuntimeWarning: invalid value encountered in divide
  normalized_X = X / X_norm
